In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

### Applications of RSA

**Identification of an individual by challenging the private key** 

The main idea behind more or less all of the following is that
1. Knowledge of a private key that "fits" a well-known public key is enough "proof" of a person's identity.
1. This knowledge needs to be proven indirectly, i.e. without revealing the private key. That wouldn't make any sense anyway, since no one knows that private key, hence no one can possibly confirm it.
  
**Here's the scenario:** Satoshi and Craig (say!) are two different persons, each with their own RSA key pair.
Everyone knows Satoshi's public key (by calling getPublic(), no problem).
Craig pretends to be Satoshi
  
We pick a secret, encrypt it with Satoshi's public key, then 
let both decrypt it and see who's doing it right  
  
*Ignore the **signString** method for the moment - we'll deal with it in the second example*

In [8]:
from RSA import Key, KeyPair
from Modular import fastExpMod
import random

class Person :
    def __init__(self):
        self._rsaKeys = KeyPair()
        
    def getPublic(self):
        return self._rsaKeys.public
    
    def decrypt(self, message):
        return self._rsaKeys.decrypt(message)
    
    def signString(self,string) :
        return string, self.decrypt(len(string)) # same as encrypt with the private key
    

# Satoshi is the "true" one and we know the public key
satoshi = Person()
satoshiKey = satoshi.getPublic()
print("I know the public key of Satoshi: ",satoshiKey)

# Craig says he's Satoshi - so, we don't need his public key, 
# he would give us Satoshi's public key anyway (everyone knows it)
craig = Person()

# We can find out who the real Satoshi is by
# 1. picking a secret message "secret"
# 2. encrypting it with Satoshi's public key
# 3. and asking both to decrypt it
#
# Only the "true" Satoshi will do that correctly!
#
secret = 1234

# Use P1's public key to encrypt the secret
challenge = fastExpMod(secret,satoshiKey.second,satoshiKey.first)
print("Challenging with",challenge)

# Let both decrypt it ==> We know who is Satoshi (the one answering with the secret)
print(satoshi.decrypt(challenge))
print(craig.decrypt(challenge))

I know the public key of Satoshi:  Key [first: 79935432919, second: 65537]
Challenging with 13708629754
1234
44710848080


**A sketch of digital signatures**

Digital signatures are metadata that can be stored along with any "document" and validate that this document has been "authored" or "approved" by a specific person.

The simplest mechanism relies on the fact that you can *use your private key to encrypt* and *your public key to decrypt* (opposite of normal, but we know that works)  
  
So a person P$_1$ (the "author") computes a property of the message (like a hash code, here it's just the length), and encrypts it with P$_1$'s *private* (!) key - that's the whole trick!  

The receiver (or "verifier") can now use P$_1$'s public key to check this property and validate that P$_1$ really signed it

In [9]:
# Two persons with their RSA keys, public parts are known
p1 = Person()
p1Key = p1.getPublic()
p2 = Person()
p2Key = p2.getPublic()

# Now P1 signs this string with length 13 encrypted with the private (!) key
# So, the encrypted length of the string is the digital signature!
string, signature = p1.signString("Hello, world!")
print("String =",string,"Signature =",signature)

# Now decrypt the signature with each of the public keys
length1 = fastExpMod(signature,p1Key.second,p1Key.first)
length2 = fastExpMod(signature,p2Key.second,p2Key.first)
# And the authenticity is clear
print("Decrypted with P1's public key:",length1,
      " \t\tAuthentic: ",len(string) == length1)
print("Decrypted with P2's public key:",length2,
      " \tAuthentic: ",len(string) == length2)

String = Hello, world! Signature = 93933982587
Decrypted with P1's public key: 13  		Authentic:  True
Decrypted with P2's public key: 544680566  	Authentic:  False


**Now we need some moment to clarify something about this text property**  

I used the length of the string as the property - for simplification purposes!  

Now, that means that an attacker could abuse this scheme:  
1. Intercept the string with the signature
1. Replace the string with another one of the same length
  
This string would now appear as being signed by P$_1$, but it's not the authentic string anymore:  
A string "Attack at dawn" could now be replaced with "Attack at noon" (note that "Attack at night" wouldn't work!) with possibly devastating consequences.
  
In fact, this scheme allows for signing, but doesn't really preserve "authenticity"!

In practice these things go hand in hand, so we need to find a string property that's not easy to fake.  

**Hashing (More in "Algorithms and Data Structures")**  

Hashing means to assign to a data object a numerical value. Different data objects might or might not have different values. The data object itself cannot be reconstructed from this value.  

Actually, the length is a kind of such a hash value, it's just not good enough to protect against faking. Strong (or "cryptographical") hash values make it practically impossible to find a string that has the same hash value as the authentic one. It's practically impossible to learn anything about the string given a strong hash.

In this example we use a cryptographical hash library called "SHA-256" (Secure Hash Algorithm with a length of 256 bit). It's considered practically secure - for even more security longer hash codes can be produced.

The runs show typical properties of strong hashes:
1. Inputs that are "close" to each other ('John' vs 'Joan') result in very different hash codes
1. Even longer inputs create hashes that just look random.



In [4]:
import hashlib

hash_object = hashlib.sha256(b'John')
hex_dig = hash_object.hexdigest()
print(hex_dig)

hash_object = hashlib.sha256(b'Joan')
hex_dig = hash_object.hexdigest()
print(hex_dig)

hash_object = hashlib.sha256(b'There now is your insular city of the Manhattoes, belted round by wharves as Indian isles by coral reefs - commerce surrounds it with her surf.')
hex_dig = hash_object.hexdigest()
print(hex_dig)

a8cfcd74832004951b4408cdb0a5dbcd8c7e52d43f7fe244bf720582e05241da
2d0f4c4eb78ce93adc09b60c696c76d0476185983c956a6f2a5bbf0afb9dbc2e
10b6271b99876bc2b3b51d76118e5a5462043f6e92a9d6d445b01735f79042e6


**Hashing for securing passwords:** You can use hash values to encode passwords:  
1. Compute the hash for the password during sign-up
2. For each login attempt compute the hash of the provided password and compare it to the sign-up version
  
Note that this is **not** foolproof, since there are definitely many strings that have the same hash code. An attacker would not have to find the correct password but any string with the same hash - that's all. These strings are called *collisions* and an attack trying to find a suiting string is called a <a href="https://en.wikipedia.org/wiki/Collision_attack">*collision attack*</a>.

Hashing passwords is the standard way of storing them safely. The main reason 
to hash passwords rather than encrypt them
is that it's even theoretically impossible to recover the original from the hash code - so even if the hash is compromised, the original password isn't!

Also, the necessity to safely store an encryption key in the system is more than tricky and dealt with <a href="https://en.wikipedia.org/wiki/Key_management">*key management*</a> techniques.

In [13]:
import hashlib

# 1. Sign up with the password
password = 's3cr3t' # terrible password
hash_object = hashlib.sha256(password.encode('utf8'))
hex_dig_original = hash_object.hexdigest()
print(hex_dig_original,"is stored")

attempt = 'passw0rd' # another terrible password
hash_object = hashlib.sha256(attempt.encode('utf8'))
hex_dig_attempt = hash_object.hexdigest()
print(hex_dig_attempt,"is attempted")

if hex_dig_original == hex_dig_attempt : 
    print("accepted!")
else :
    print("rejected!")

4e738ca5563c06cfd0018299933d58db1dd8bf97f6973dc99bf6cdc64b5550bd is stored
8f0e2f76e22b43e2855189877e7dc1e1e7d98c226c95db247cd1d547928334a9 is attempted
rejected!


We "improve" the previous example with a weak hash that just adds the decimal value of the characters together. It's still weak, but it makes it much more difficult to sneak an unauthenticated replacement in - it needs to have the same "hash", not be complete gibberish and express something malicious.  

The reason I'm not using SHA here is that this toy implementaion of RSA cannot handle numbers that big. You certainly can work around that, by encrypting the string character character and somehow magically appending this into a value. 

In [11]:
class SecurePerson (Person):
    @classmethod
    def hashStringBadly(cls,string) :
        sum = 0
        for c in string :
            sum += ord(c)
        return sum
    
    def signString(self,string) :
        return string, self.decrypt(SecurePerson.hashStringBadly(string)) # same as encrypt with the private key
    
# Same with safe persons:
p1 = SecurePerson()
p1Key = p1.getPublic()
p2 = SecurePerson()
p2Key = p2.getPublic()

string, signature = p1.signString("Hello, world!")
print("String =",string,"Signature =",signature)

# Now decrypt the signature with each of the public keys
hash1 = fastExpMod(signature,p1Key.second,p1Key.first)
hash2 = fastExpMod(signature,p2Key.second,p2Key.first)
# And the authenticity is clear
print("Decrypted with P1's public key:",hash1,
      " \t\tAuthentic: ",SecurePerson.hashStringBadly(string) == hash1)
print("Decrypted with P2's public key:",hash2,
      " \tAuthentic: ",SecurePerson.hashStringBadly(string) == hash2)

String = Hello, world! Signature = 27991652002
Decrypted with P1's public key: 1161  		Authentic:  True
Decrypted with P2's public key: 9593639908  	Authentic:  False


## A look at the standard RSA package in Python  

Nothing too special, one little thing that's not textbook: Public and private key are NOT interchangeable! The reason is (I assume) that the decryption method checks if the cyphertext has been properly encrypted with the public key (which is a part of the private key, if I see that correctly).  

That means you cannot really use it for Digital Signatures the way it was sketched, requiring a separate functionality to do that.

In [14]:
import rsa

# Basic RSA encryption/decryption

# Key pair generation
(pubkey, privkey) = rsa.newkeys(2048) # Pick 2048 bit key length
print(pubkey,"\n",privkey,"\n\n")

# Encryption/Decryption
message = 'Hello, World!'.encode('utf8')
crypto = rsa.encrypt(message, pubkey)
print("Cyphertext:",crypto)
decrypt = rsa.decrypt(crypto, privkey)
print(message.decode('utf8'),"\n\n")

# Digital signature is extra, because (apparently) public and private key are not interchangeable
# Important is that the message is signed with the private key (as it should)
signature = rsa.sign(message, privkey, 'SHA-256')

# Verify with the public key!
print("Signature:",signature,"\nVerified as",rsa.verify(message, signature, pubkey))

# Wrong signature: Have to catch the exception (OHHH-KAYYY)
signature = "12345".encode('utf-8')
try:
    print("Signature:",signature,"\nVerified as",rsa.verify(message, signature, pubkey))
except rsa.VerificationError:
    print("Invalid signature")


PublicKey(19951795933870193438847590185398362324570905865922635322438638442729844508742680230295540887852217347225020388990393251427630317609407410286363709746727225477926865936378302453197757115546964997341573636534804493967139991206301323718201188234221442183332589802089621328046217743562672986106778697344118160158444348841928161958431735393243676733357122680589611965346179975925784305775218070333404046412866438942703090921174060337777387671135878879764719438941110536071740476738809211105360130578299592059609002348011699032066149109534760926261234438974901554337426207843180760183812740058468094681263848330580837322847, 65537) 
 PrivateKey(199517959338701934388475901853983623245709058659226353224386384427298445087426802302955408878522173472250203889903932514276303176094074102863637097467272254779268659363783024531977571155469649973415736365348044939671399912063013237182011882342214421833325898020896213280462177435626729861067786973441181601584443488419281619584317353932436767333571226

**Here's the improved Person - not a subclass, since it redefines everything**

In [15]:
import rsa

class SecurestPerson :
    def __init__(self):
        (self._pubkey, self._privkey) = rsa.newkeys(2048)
        
    def getPublic(self):
        return self._pubkey
    
    def decrypt(self, message):
        return rsa.decrypt(message,self._privkey)
    
    def signString(self,string) :
        return string, rsa.sign(string, self._privkey, 'SHA-256')

**That's now the Satoshi example again**  

Slightly recoded to fit the RSA API, but really essentiaqlly the same!

In [16]:
import rsa

# Satoshi
satoshi = SecurestPerson()
satoshiKey = satoshi.getPublic()
print("I know the public key of Satoshi: ",satoshiKey)

# Craig
craig = SecurestPerson()

# Secret message
secret = "1234".encode('utf8')

# Use P1's public key to encrypt the secret
challenge = rsa.encrypt(secret, satoshiKey)
print("Challenging with",challenge)

# Let both decrypt it ==> We know who is Satoshi (the one answering with the secret)
try:
    print(satoshi.decrypt(challenge).decode('utf8'))
except rsa.DecryptionError :
    print("Satoshi cannot decrypt!")
try:
    print(craig.decrypt(challenge).decode('utf8'))
except rsa.DecryptionError :
    print("Craig cannot decrypt!")


I know the public key of Satoshi:  PublicKey(16402014677862766688798580147641488147505729345300840973490840596700887160956662690419884866733058852162936640111719502365323270221253655786074239329157067735943523089182169186198836974775567059352741144346171943729010112387123927779231869006250966800276174769324910008577041802332117608339904615523893652750274176121348034302257126977840189665566935993625034566072890310906741223745729443461493336850411803667477706156564698334626549041092842794871633374165344706581199111941938820988969884910056298466671827792349673265157282520589647000830696481521565854471870227596780997805141951177270169629698429268385099138943, 65537)
Challenging with b"n\xcal\x00\xd8 Pk^\x03\\O\x01\x95\xc9\xbdS\xe6\xb1\x9ajH,\xce\x9cm\x880\xdbzd\x86\xbbF#Ym\xbf`\xf3l\xe5\xc8\xdd0h\xf6g\xf7\xe8\xaa\xca\xbfak\xa7+\xc1\x04\xefU\xb9\xf27V\xbd\xc5X\x95\xd6\x11\xfe\x84'\x14/Oq(Ju;\xcf\xf4CT\x9e\x95\x8f]0\x0b\x8a\x00\xbc{\x90\xeeg\xf2\xc9\x1cH\xb1\x1b\xa3\xdd\x01V\xd9\x13\xfb$?\x9